In [1]:
from main import import_from_gsheet
merged_df = import_from_gsheet()

In [7]:
merged_df.columns = ['lemma', 'zh-def', 'syllable', 'ori-def', 'ori-page', 'comment', 'annotator']
merged_df

,lemma,zh-def,syllable,ori-def,ori-page,comment,annotator
0,Aûrras,一步,A-ûrras,"a step, a pace from 標示不清",6,,妤蓁_13-41
1,Ailijt,粗草的莖,A-ilijt,the stalk or stem of coarse grass,6,,妤蓁_13-41
2,Aomo,女士的髮帶,A-omo,a lady's fillet for the hair,6,,妤蓁_13-41
3,Abak,小船,Abak,a small boat or sampan,6,,妤蓁_13-41
4,Abûs,海,Abûs,the sea,6,,妤蓁_13-41
...,...,...,...,...,...,...,...
3084,Tsikarra,長尾巴的小田蜥蜴,Tsi_kar_ra,"smaller ditto, with a long tail",360,,峻維_326-360
3085,Tsipollopollul,有斑點的房子蜥蜴,Tsi_pol_lo_pol_lul,a spotted house lizard,360,,峻維_326-360
3086,Tsipan,西方,Tsi_pan,the west,360,,峻維_326-360
3087,Tsiotsio,時鐘,Tsio_tsio,"a clock, or bell",360,,峻維_326-360


In [10]:
# Save to JSON
merged_df[['lemma', 'zh-def', 'ori-def', 'ori-page']].to_json("docs/dict.json", force_ascii=False)
merged_df[['lemma', 'zh-def', 'ori-def', 'ori-page']].to_csv("docs/dict.csv")

In [11]:
import os
os.listdir("docs")

['Favorlang_eng.pdf',
 'dict.json',
 'index.html',
 'dict.csv',
 '.ipynb_checkpoints',
 'favorlang_dict_transcribed.pdf',
 'style.css']

In [14]:
import pandas as pd


merged_df['lemma'] = pd.core.strings.str_strip(merged_df['lemma'])

In [17]:
for col in merged_df.columns:
    merged_df[col] = pd.core.strings.str_strip(merged_df[col])

0               Aûrras
1               Ailijt
2                 Aomo
3                 Abak
4                 Abûs
             ...      
3084          Tsikarra
3085    Tsipollopollul
3086            Tsipan
3087          Tsiotsio
3088               Tsi
Name: lemma, Length: 3089, dtype: object
0             一步
1           粗草的莖
2          女士的髮帶
3             小船
4              海
          ...   
3084    長尾巴的小田蜥蜴
3085    有斑點的房子蜥蜴
3086          西方
3087          時鐘
3088           魚
Name: zh-def, Length: 3089, dtype: object
0                 A-ûrras 
1                  A-ilijt
2                    A-omo
3                     Abak
4                     Abûs
               ...        
3084            Tsi_kar_ra
3085    Tsi_pol_lo_pol_lul
3086               Tsi_pan
3087             Tsio_tsio
3088                   Tsi
Name: syllable, Length: 3089, dtype: object
0                                a step, a pace from 標示不清
1                       the stalk or stem of coarse grass
2                   